# スーパーマーケット商品データの表記揺れ正規化（SQL版）

このノートブックでは、Databricks SQL の `ai_query()` 関数を使用して、商品名の表記揺れをLLMバッチ推論で正規化します。

## 概要
- **カタログ/スキーマ**: `users.yukiteru_koide`
- **LLMエンドポイント**: `databricks-gpt-oss-120b`
- **目的**: 商品名の表記揺れ（カタカナ、英語、略称など）を統一された正式名称に変換
- **使用言語**: SQL のみ


In [0]:
-- Step 1: カタログとスキーマの作成
CREATE CATALOG IF NOT EXISTS users;
CREATE SCHEMA IF NOT EXISTS users.yukiteru_koide;


In [0]:
-- Step 2: サンプルテーブルの作成（表記揺れのある商品データ）
DROP TABLE IF EXISTS users.yukiteru_koide.products_with_variants;

CREATE TABLE users.yukiteru_koide.products_with_variants (
    product_id INT,
    product_name STRING,
    category STRING,
    price INT
);


In [0]:
-- Step 3: サンプルデータの挿入
INSERT INTO users.yukiteru_koide.products_with_variants VALUES
  (1, 'ﾍﾟｯﾄﾎﾞﾄﾙ緑茶', '飲料', 150),
  (2, 'pet bottle green tea', '飲料', 150),
  (3, '緑茶ペット', '飲料', 150),
  (4, 'ｺｶｺｰﾗ', '飲料', 140),
  (5, 'Coca Cola', '飲料', 140),
  (6, 'コーラ', '飲料', 140),
  (7, 'ﾎﾟﾃﾄﾁｯﾌﾟｽ', 'スナック', 120),
  (8, 'potato chips', 'スナック', 120),
  (9, 'ポテチ', 'スナック', 120),
  (10, 'ｶｯﾌﾟﾗｰﾒﾝ', '食品', 180),
  (11, 'cup ramen', '食品', 180),
  (12, 'カップ麺', '食品', 180),
  (13, 'ﾁｮｺﾚｰﾄ', '菓子', 200),
  (14, 'chocolate', '菓子', 200),
  (15, 'チョコ', '菓子', 200);


In [0]:
-- Step 4: 元データの確認
SELECT * FROM users.yukiteru_koide.products_with_variants
ORDER BY product_id;


## ai_query() を使用したLLMバッチ推論

Databricks SQL の `ai_query()` 関数を使用して、商品名の表記揺れを正規化します。

### プロンプト設計
- 入力: 表記揺れのある商品名
- 出力: 統一された正式な商品名（日本語カタカナ表記）


In [0]:
-- Step 5: ai_query() を使ってLLMバッチ推論を実行し、正規化テーブルを作成
CREATE OR REPLACE TABLE users.yukiteru_koide.products_normalized AS
SELECT 
    product_id,
    product_name AS original_name,
    category,
    price,
    ai_query(
        'databricks-gpt-oss-120b',
        CONCAT(
            '次の商品名を統一された正式な商品名に正規化してください。',
            '日本語のカタカナまたは一般的な表記で、商品名のみを簡潔に回答してください。',
            '説明や補足は不要です。',
            '商品名: ', product_name
        )
    ) AS normalized_name
FROM users.yukiteru_koide.products_with_variants;


In [0]:
-- Step 6: 正規化結果の確認
SELECT 
    product_id,
    original_name,
    normalized_name,
    category,
    price
FROM users.yukiteru_koide.products_normalized
ORDER BY product_id;


## 正規化結果の分析

表記揺れがどのように統一されたかを分析します。


In [0]:
-- Step 7: 表記揺れのグループ化分析
SELECT 
    normalized_name,
    category,
    COUNT(*) AS variant_count,
    COLLECT_LIST(original_name) AS original_variants,
    ROUND(AVG(price), 2) AS avg_price
FROM users.yukiteru_koide.products_normalized
GROUP BY normalized_name, category
ORDER BY variant_count DESC, normalized_name;


In [0]:
-- Step 8: 統計情報の表示
SELECT 
    '元の商品数' AS metric,
    COUNT(*) AS value
FROM users.yukiteru_koide.products_with_variants

UNION ALL

SELECT 
    '正規化後のユニーク商品数' AS metric,
    COUNT(DISTINCT normalized_name) AS value
FROM users.yukiteru_koide.products_normalized

UNION ALL

SELECT 
    '削減率 (%)' AS metric,
    ROUND(
        (1 - CAST(COUNT(DISTINCT normalized_name) AS DOUBLE) / 
         (SELECT COUNT(*) FROM users.yukiteru_koide.products_with_variants)) * 100,
        1
    ) AS value
FROM users.yukiteru_koide.products_normalized;


## まとめ

### 実施内容
1. ✅ カタログ・スキーマの作成
2. ✅ 表記揺れのある商品データテーブル作成 (`users.yukiteru_koide.products_with_variants`)
3. ✅ `ai_query()` 関数でLLMバッチ推論を実行
4. ✅ 正規化テーブル作成 (`users.yukiteru_koide.products_normalized`)
5. ✅ 表記揺れの統一結果を分析

### 使用技術
- **言語**: SQL のみ
- **LLMエンドポイント**: `yukiteru_LLM`
- **Databricks関数**: `ai_query()`
- **データ処理**: Delta Lake

### 期待される結果
- 半角カタカナ → 全角カタカナ
- 英語表記 → 日本語表記
- 略称 → 正式名称
- 15個の表記揺れ → 5個のユニーク商品に集約

### 次のステップ
- プロンプトの最適化で精度向上
- 大規模データへの適用
- Databricks Workflows で定期実行
- マスタデータとの突合


---

## 参考情報

### ai_query() 関数の使い方

```sql
ai_query(
    'エンドポイント名',  -- 例: 'yukiteru_LLM'
    'プロンプト文字列'
)
```

### エンドポイントURL
- エンドポイント名: `yukiteru_LLM`
- 完全URL: `https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/yukiteru_LLM/invocations`
- `ai_query()` 関数ではエンドポイント名を指定するだけで自動的にURLが解決されます

### プロンプトのベストプラクティス
1. 明確な指示を与える
2. 出力形式を具体的に指定
3. 不要な説明を排除するよう指示
4. テストして精度を確認し、改善する
